In [1]:
import theano
from theano.sandbox import cuda
theano.sandbox.cuda.use("gpu0")

/home/irashadow/.pyenv/versions/miniconda-3.9.1/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:541: UserWarning: Theano flag device=gpu* (old gpu back-end) only support floatX=float32. You have floatX=float64. Use the new gpu back-end with device=cuda* for that value of floatX.
  warnings.warn(msg)
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 4007)


In [91]:
%matplotlib inline

import model; reload(model)
from model import FullyConvolutionalNetwork

from preprocess import *

import h5py

from keras.optimizers import Adam
from keras import backend as K

In [92]:
img_size = 224
nb_class = 21
path_to_train = 'dataset'
path_to_target = 'dataset'
path_to_txt = '/home/irashadow/python_workspace/FCN_via_keras/train0.txt'

#Keras parameters
weight = ""
epoch = 20
batchsize = 1
lr = 1e-3



In [93]:
with open(path_to_txt,"r") as f:
    ls = f.readlines()
names = [l.rstrip('\n') for l in ls]
nb_data = len(names)
print nb_data

5


# FCN Model

In [ ]:
from keras.layers import merge, Lambda, Convolution2D, Deconvolution2D, MaxPooling2D, Input, Reshape, Permute, ZeroPadding2D, UpSampling2D, Cropping2D
from keras.layers.core import Activation
from keras.models import Model
from keras.engine.topology import Layer
from keras.utils.layer_utils import layer_from_config
from keras.utils import np_utils, generic_utils
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input

import cv2
import numpy as np

class Softmax2D(Layer):
    def __init__(self, **kwargs):
        super(Softmax2D, self).__init__(**kwargs)

    def build(self, input_shape):
        pass

    def call(self, x, mask=None):
        e = K.exp(x - K.max(x, axis=-1, keepdims=True))
        s = K.sum(e, axis=-1, keepdims=True)
        return K.clip(e/s, 1e-7, 1)

    def get_output_shape_for(self, input_shape):
        return (input_shape)
    
    

In [77]:
vgg16 = VGG16(include_top=False,
              weights='imagenet',
              input_tensor=None,
              input_shape=(3,224,224))

In [78]:
ip = Input(shape=(3,img_size, img_size))
h = vgg16.layers[1](ip)
h = vgg16.layers[2](h)
h = vgg16.layers[3](h)
h = vgg16.layers[4](h)
h = vgg16.layers[5](h)
h = vgg16.layers[6](h)
h = vgg16.layers[7](h)
h = vgg16.layers[8](h)
h = vgg16.layers[9](h)
h = vgg16.layers[10](h)

In [79]:
#split layer
p3 = h
p3 = Convolution2D(nb_class, 1, 1, activation='relu', border_mode='valid')(p3)

#(21*28*28)
h = vgg16.layers[11](h)
h = vgg16.layers[12](h)
h = vgg16.layers[13](h)
h = vgg16.layers[14](h)

In [80]:
#split layer
p4 = h
p4 = Convolution2D(nb_class, 1, 1, activation='relu')(p4)
"""
p4 = Deconvolution2D(self.FCN_CLASSES, 4, 4,
        output_shape=(self.batchsize, self.img_height//8, self.img_width//8, self.FCN_CLASSES),
        subsample=(2, 2),
        border_mode='valid')(p4)
"""
p4 = UpSampling2D((2,2))(p4)
p4 = Convolution2D(nb_class, 3, 3, activation='relu', border_mode='same')(p4)

h = vgg16.layers[15](h)
h = vgg16.layers[16](h)
h = vgg16.layers[17](h)
h = vgg16.layers[18](h)

In [81]:
p5 = h
p5 = Convolution2D(nb_class, 1, 1, activation='relu')(p5)
"""
p5 = Deconvolution2D(self.FCN_CLASSES, 8, 8,
        output_shape=(self.batchsize, self.img_height//8, self.img_width//8, self.FCN_CLASSES),
        subsample=(4, 4),
        border_mode='valid')(p5)
"""
p5 = UpSampling2D((4, 4))(p5)
p5 = Convolution2D(nb_class, 3, 3, activation='relu', border_mode='same')(p5)

In [ ]:
# merge scores
h = merge([p3, p4, p5], mode="sum")

"""
h = Deconvolution2D(self.FCN_CLASSES, 16, 16,
        output_shape=(self.batchsize, 224, 224, self.FCN_CLASSES),
        subsample=(8, 8),
        border_mode='valid')(h)
"""
h = UpSampling2D((8, 8))(h)
h = Convolution2D(nb_class, 3, 3, activation='relu', border_mode='same')(h)

In [ ]:
h = Reshape((nb_class,img_size*img_size))(h)
h = Permute((2,1))(h)

In [89]:
out = Activation("softmax")(h)

In [90]:
model_temp = Model(ip, out)
model_temp.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_16 (InputLayer)            (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 64, 224, 224)  1792        input_16[0][0]                   
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 64, 224, 224)  36928       block1_conv1[1][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, 112, 112)  0           block1_conv2[1][0]               
___________________________________________________________________________________________

In [94]:
FCN = FullyConvolutionalNetwork(img_height=img_size, img_width=img_size, FCN_CLASSES=nb_class)

In [97]:
adam = Adam(lr=lr)
train_model = FCN.create_model(train_flag=True)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_18 (InputLayer)            (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 64, 224, 224)  1792        input_18[0][0]                   
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 64, 224, 224)  36928       block1_conv1[1][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, 112, 112)  0           block1_conv2[1][0]               
___________________________________________________________________________________________

In [98]:
train_model.compile(loss="categorical_crossentropy", optimizer='adam')
if len(weight):
    model.load_weights(weight, model)
print("Num data: {}".format(nb_data))

Num data: 5
